<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_8_tuning_cadec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0    989      0 --:--:-- --:--:-- --:--:--   989
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-8.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [12]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-8.validation.csv",header=None)
valid.head()

,0,1
0,0,scared to death
1,2,heel pain
2,5,increased heart rate
3,6,stage 1 diastolic dysfunction
4,8,arthritis


In [13]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-8.test.csv",header=None)
test.head()

,0,1
0,1,Abdominal rash
1,5,increase my heart rate
2,5,tachycardia
3,8,arthritis
4,8,arthritis


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine_tuning_language_model_cadec_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [0]:
data_lm.save()
data_clas.save()

In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.883290,4.180188,0.222481,03:40


In [0]:
# learn.fit_one_cycle(1, 1e-2)

In [0]:
# learn.fit_one_cycle(1, 1e-3)

In [0]:
# learn.fit_one_cycle(1, 1e-3)

In [0]:
learn.save_encoder('ft_fold8_first')

In [27]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_fold8_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.043207,3.749289,0.364982,01:09


In [28]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.098938,3.212894,0.466091,01:19


In [29]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.587978,2.855953,0.499383,03:12


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.196553,2.426138,0.581998,03:24
1,1.873008,2.265628,0.609125,03:08


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.715652,2.028778,0.633785,03:17
1,1.521536,1.911588,0.654747,03:17


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.445386,1.820033,0.674476,03:31
1,1.267859,1.693510,0.696671,03:11


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.229625,1.620967,0.694205,03:25
1,1.139120,1.543079,0.713933,03:10


In [34]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.074374,1.464134,0.725031,03:22
1,0.975621,1.428230,0.727497,03:12


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.924067,1.413153,0.737361,03:19
1,0.926870,1.406281,0.747226,03:17
2,0.792982,1.342943,0.764488,03:18
3,0.750116,1.304390,0.763255,03:29


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.738582,1.358660,0.747226,03:26
1,0.784809,1.304663,0.770653,03:28
2,0.695618,1.286613,0.778052,03:17
3,0.647079,1.293618,0.778052,03:20


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.633375,1.299158,0.781751,03:17
1,0.714634,1.305762,0.769420,03:13
2,0.665350,1.252259,0.789149,03:11
3,0.606639,1.301398,0.787916,03:05


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.591225,1.321991,0.790382,03:05
1,0.621756,1.339512,0.786683,03:10
2,0.589414,1.274100,0.792848,03:27
3,0.525143,1.256550,0.791615,03:21


In [0]:
# save the best model

learn.save_encoder('ask-8-tuning-cadec')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [41]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[34, 379, 455, 8, 8, 8, 126, 20, 500, 27, 491, 491, 31, 31, 34, 880, 76, 741, 37, 37, 37, 896, 42, 928, 49, 775, 49, 49, 775, 49, 49, 53, 53, 53, 53, 53, 962, 597, 597, 265, 74, 76, 76, 79, 79, 614, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 260, 614, 265, 80, 80, 80, 80, 85, 1031, 91, 455, 455, 94, 94, 94, 94, 1031, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 289, 635, 94, 234, 1031, 100, 100, 100, 100, 100, 100, 100, 352, 105, 660, 660, 289, 1035, 111, 111, 111, 111, 111, 111, 111, 111, 289, 289, 1032, 127, 290, 777, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 513, 490, 127, 355, 94, 880, 243, 406, 137, 137, 139, 142, 847, 151, 151, 1031, 157, 159, 1035, 165, 166, 350, 58, 289, 289, 289, 864, 877, 724, 384, 286, 182, 182, 182, 182, 289, 289, 183, 183, 183, 183, 183, 183, 183, 183, 184, 184, 872, 265, 187, 187, 187, 402, 189,

In [42]:
correct_num = 0

for i in range(0, 866):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

530
0.6120092378752887
